# Advanced Machine Learning: MLE, NLL, MAP, and the Relationship with MSE and MAE

## Important Note:
**This notebook is not intended for beginners.** It is designed for upper intermediate machine learning learners who are seeking a deep understanding of Maximum Likelihood Estimation (MLE), Negative Log-Likelihood (NLL), Maximum A Posteriori Estimation (MAP), and how they relate to common loss functions such as Mean Squared Error (MSE) and Mean Absolute Error (MAE). We will explore these concepts with rigorous mathematical explanations and detailed Python implementations from scratch.

# 1. Introduction to Estimation Theory
Estimation theory lies at the intersection of statistics and machine learning, where the goal is to infer unknown parameters of probabilistic models from observed data. We focus on two major estimation paradigms:

1. **Maximum Likelihood Estimation (MLE)**: A frequentist approach based on maximizing the probability of observed data.
2. **Maximum A Posteriori Estimation (MAP)**: A Bayesian method that incorporates prior knowledge about the parameters into the estimation process.

These techniques are foundational in machine learning, underpinning many algorithms from linear regression to deep learning. In this notebook, we will derive these estimators mathematically, analyze their properties, and relate them to commonly used loss functions like MSE and MAE.


# 2. Maximum Likelihood Estimation (MLE)
Maximum Likelihood Estimation is a fundamental method for parameter estimation in statistics. It is based on the principle of choosing the parameter values that maximize the likelihood of the observed data under the assumed model.

### 2.1 Mathematical Formulation
Let $X = \{x_1, x_2, \dots, x_n\}$ be an independent and identically distributed (i.i.d.) sample from a probability distribution with parameter $\theta$. The likelihood function is:

$$ L(\theta; X) = \prod_{i=1}^{n} p(x_i | \theta) $$

The log-likelihood function, which simplifies the product to a sum, is given by:

$$ \log L(\theta; X) = \sum_{i=1}^{n} \log p(x_i | \theta) $$

The MLE $\hat{\theta}_{MLE}$ is the value of $\theta$ that maximizes the log-likelihood:

$$ \hat{\theta}_{MLE} = \arg\max_{\theta} \log L(\theta; X) $$

### 2.2 Derivation for Normal Distribution
Consider the normal distribution with unknown mean $\mu$ and variance $\sigma^2$. The probability density function (PDF) is:

$$ p(x_i | \mu, \sigma^2) = \frac{1}{\sqrt{2\pi\sigma^2}} \exp\left(-\frac{(x_i - \mu)^2}{2\sigma^2}\right) $$

The log-likelihood for a sample $X$ is:

$$ \log L(\mu, \sigma^2; X) = -\frac{n}{2} \log(2\pi\sigma^2) - \frac{1}{2\sigma^2} \sum_{i=1}^{n} (x_i - \mu)^2 $$

To find the MLE, we differentiate this log-likelihood with respect to $\mu$ and $\sigma^2$, set the derivatives to zero, and solve for the parameters.

### 2.3 Proof of MLE Consistency
MLE is consistent under regularity conditions, meaning that as $n \to \infty$, the MLE converges in probability to the true parameter value $\theta_0$. This is formalized by the law of large numbers applied to the score function (the derivative of the log-likelihood). The detailed proof involves showing that the Fisher Information matrix is positive definite and that the likelihood function is asymptotically normal.


In [1]:
import numpy as np
from scipy.stats import norm

class MLENormal:
    def __init__(self, data):
        self.data = np.array(data)
    
    def log_likelihood(self, mu, sigma2):
        """Compute the log-likelihood for normal distribution."""
        n = len(self.data)
        ll = -n/2 * np.log(2 * np.pi * sigma2) - np.sum((self.data - mu)**2) / (2 * sigma2)
        return ll
    
    def score_function(self, mu, sigma2):
        """Compute the score function (gradient of log-likelihood)."""
        n = len(self.data)
        score_mu = np.sum(self.data - mu) / sigma2
        score_sigma2 = -n / (2 * sigma2) + np.sum((self.data - mu)**2) / (2 * sigma2**2)
        return score_mu, score_sigma2
    
    def fisher_information(self, sigma2):
        """Compute the Fisher information matrix for normal distribution."""
        n = len(self.data)
        fisher_mu2 = n / sigma2
        fisher_sigma2_2 = n / (2 * sigma2**2)
        return np.array([[fisher_mu2, 0], [0, fisher_sigma2_2]])

# Example usage
data = [2.3, 2.5, 3.1, 3.8, 2.7]
mle = MLENormal(data)

mu_initial, sigma2_initial = np.mean(data), np.var(data)
log_likelihood = mle.log_likelihood(mu_initial, sigma2_initial)
score_mu, score_sigma2 = mle.score_function(mu_initial, sigma2_initial)
fisher_matrix = mle.fisher_information(sigma2_initial)

print(f"Initial Log-Likelihood: {log_likelihood}")
print(f"Score Function (mu, sigma^2): ({score_mu}, {score_sigma2})")
print(f"Fisher Information Matrix:\n{fisher_matrix}")


Initial Log-Likelihood: -3.926523529277739
Score Function (mu, sigma^2): (1.5770213417970976e-15, 1.7763568394002505e-15)
Fisher Information Matrix:
[[17.75568182  0.        ]
 [ 0.         31.52642368]]


# 3. Negative Log-Likelihood (NLL)
The Negative Log-Likelihood (NLL) is the negative of the log-likelihood and is often used as a loss function in machine learning, particularly in models that predict probabilities. Minimizing the NLL corresponds to maximizing the likelihood.

For a normal distribution, the NLL is:

$$ \text{NLL}(\mu, \sigma^2) = \frac{n}{2} \log(2\pi\sigma^2) + \frac{1}{2\sigma^2} \sum_{i=1}^{n} (x_i - \mu)^2 $$

The NLL is derived by taking the negative of the log-likelihood, and it represents the cost function that we minimize in many probabilistic models.

### 3.1 Gradient of the NLL
To perform gradient-based optimization, we need the gradient (or score function) of the NLL with respect to the parameters. For the normal distribution:

$$ \frac{\partial \text{NLL}}{\partial \mu} = -\frac{1}{\sigma^2} \sum_{i=1}^{n} (x_i - \mu) $$

$$ \frac{\partial \text{NLL}}{\partial \sigma^2} = \frac{n}{2\sigma^2} - \frac{1}{2\sigma^4} \sum_{i=1}^{n} (x_i - \mu)^2 $$


In [2]:
# NLL calculation and gradient implementation
def negative_log_likelihood(mu, sigma2, data):
    n = len(data)
    nll = n / 2 * np.log(2 * np.pi * sigma2) + np.sum((data - mu)**2) / (2 * sigma2)
    return nll

def nll_gradient(mu, sigma2, data):
    n = len(data)
    grad_mu = -np.sum(data - mu) / sigma2
    grad_sigma2 = n / (2 * sigma2) - np.sum((data - mu)**2) / (2 * sigma2**2)
    return grad_mu, grad_sigma2

nll_value = negative_log_likelihood(mu_initial, sigma2_initial, data)
grad_mu, grad_sigma2 = nll_gradient(mu_initial, sigma2_initial, data)

print(f"NLL Value: {nll_value}")
print(f"NLL Gradient (mu, sigma^2): ({grad_mu}, {grad_sigma2})")


NLL Value: 3.926523529277739
NLL Gradient (mu, sigma^2): (-1.5770213417970976e-15, -1.7763568394002505e-15)


# 4. Maximum A Posteriori Estimation (MAP)

### 4.1 Derivation of MAP for Normal Distribution with Gaussian Prior
Given a dataset $X = \{x_1, x_2, \dots, x_n\}$, suppose we assume that the data is normally distributed with unknown mean $\mu$ and known variance $\sigma^2$. Additionally, we assume a Gaussian prior on $\mu$ with mean $\mu_0$ and variance $\sigma_0^2$:

$$ P(\mu) \propto \exp\left(-\frac{(\mu - \mu_0)^2}{2\sigma_0^2}\right) $$

The posterior distribution, according to Bayes' theorem, is proportional to the product of the likelihood and the prior:

$$ P(\mu | X) \propto \exp\left(-\frac{n}{2\sigma^2} (\mu - \bar{x})^2 - \frac{(\mu - \mu_0)^2}{2\sigma_0^2}\right) $$

Taking the logarithm of the posterior (log-posterior):

$$ \log P(\mu | X) = -\frac{n}{2\sigma^2} (\mu - \bar{x})^2 - \frac{(\mu - \mu_0)^2}{2\sigma_0^2} + C $$

Where $C$ is a constant that does not depend on $\mu$. To find the MAP estimate, we differentiate the log-posterior with respect to $\mu$, set the derivative to zero, and solve for $\mu$:

$$ \hat{\mu}_{MAP} = \frac{\sigma^2 \mu_0 + n\sigma_0^2 \bar{x}}{n\sigma_0^2 + \sigma^2} $$

This is the MAP estimate for $\mu$, which is a weighted average of the prior mean $\mu_0$ and the sample mean $\bar{x}$, with weights depending on the variances of the prior and the data.

### 4.2 Interpretation of MAP
The MAP estimator introduces regularization into the estimation process by incorporating prior information. When the prior variance $\sigma_0^2$ is large (i.e., weak prior belief), the MAP estimate approaches the MLE. Conversely, when the prior variance is small (i.e., strong prior belief), the MAP estimate is biased towards the prior mean $\mu_0$. This regularization effect is particularly useful in situations where the data is sparse, and prior knowledge can guide the estimation process.


In [3]:
import numpy as np

def map_estimation(data, mu_prior, sigma2_prior, sigma2_data):
    """
    MAP estimate for the mean of a normal distribution with a Gaussian prior.
    
    Args:
    - data: Array of data points.
    - mu_prior: Mean of the Gaussian prior on mu.
    - sigma2_prior: Variance of the Gaussian prior on mu.
    - sigma2_data: Variance of the data (assumed known).
    
    Returns:
    - mu_map: The MAP estimate of mu.
    """
    n = len(data)
    sample_mean = np.mean(data)
    
    # MAP estimate formula
    mu_map = (sigma2_data * mu_prior + n * sigma2_prior * sample_mean) / (n * sigma2_prior + sigma2_data)
    return mu_map

# Example data and prior parameters
data = np.array([2.3, 2.5, 2.7, 2.9, 3.1])  # Replace with your actual data points
mu_prior = 2.0         # Mean of the Gaussian prior
sigma2_prior = 0.5     # Variance of the Gaussian prior
sigma2_data = 1.0      # Known variance of data

# Calculate the MAP estimate
mu_map_estimate = map_estimation(data, mu_prior, sigma2_prior, sigma2_data)

print(f"MAP estimate for mu: {mu_map_estimate}")


MAP estimate for mu: 2.5


# 5. Relationship Between MSE, MAE, and Likelihoods

### 5.1 Mean Squared Error (MSE) and Gaussian Likelihood
Mean Squared Error (MSE) is a commonly used loss function in regression models. It can be derived from the assumption that the errors (residuals) are normally distributed. Suppose the model predictions are $f(x_i)$ and the true values are $y_i$, then the residuals $r_i = y_i - f(x_i)$ follow a normal distribution:

$$ r_i \sim \mathcal{N}(0, \sigma^2) $$

The likelihood of the residuals is:

$$ L(\sigma^2; r) = \prod_{i=1}^{n} \frac{1}{\sqrt{2\pi\sigma^2}} \exp\left(-\frac{r_i^2}{2\sigma^2}\right) $$

The negative log-likelihood (NLL) for this Gaussian model is:

$$ \text{NLL} = \frac{n}{2} \log(2\pi\sigma^2) + \frac{1}{2\sigma^2} \sum_{i=1}^{n} r_i^2 $$

Minimizing the NLL with respect to $\sigma^2$ is equivalent to minimizing the MSE:

$$ \text{MSE} = \frac{1}{n} \sum_{i=1}^{n} r_i^2 $$

### 5.2 Mean Absolute Error (MAE) and Laplace Likelihood
Mean Absolute Error (MAE) is another common loss function, particularly when robustness to outliers is desired. It corresponds to assuming that the residuals follow a Laplace distribution:

$$ r_i \sim \text{Laplace}(0, b) $$

The likelihood of the residuals is:

$$ L(b; r) = \prod_{i=1}^{n} \frac{1}{2b} \exp\left(-\frac{|r_i|}{b}\right) $$

The negative log-likelihood for this model is:

$$ \text{NLL} = n \log(2b) + \frac{1}{b} \sum_{i=1}^{n} |r_i| $$

Minimizing this NLL with respect to $b$ is equivalent to minimizing the MAE:

$$ \text{MAE} = \frac{1}{n} \sum_{i=1}^{n} |r_i| $$

### 5.3 Connection to NLL
Both MSE and MAE can be viewed as special cases of the negative log-likelihood (NLL). MSE corresponds to assuming Gaussian noise, while MAE corresponds to assuming Laplace noise. By framing regression problems in this probabilistic context, we can better understand the assumptions underlying different loss functions and choose the appropriate model for the given data.


In [4]:
# MSE calculation and Gaussian likelihood implementation
def mean_squared_error(y_true, y_pred):
    """Compute Mean Squared Error (MSE)."""
    return np.mean((y_true - y_pred)**2)

def gaussian_likelihood(residuals, sigma2):
    """Compute the Gaussian likelihood of residuals."""
    n = len(residuals)
    likelihood = (1 / np.sqrt(2 * np.pi * sigma2)) ** n * np.exp(-np.sum(residuals**2) / (2 * sigma2))
    return likelihood

# Example data
y_true = np.array([2.3, 2.5, 3.1, 3.8, 2.7])
y_pred = np.array([2.4, 2.6, 3.0, 3.7, 2.8])
residuals = y_true - y_pred

# MSE and Gaussian likelihood
mse_value = mean_squared_error(y_true, y_pred)
gaussian_likelihood_value = gaussian_likelihood(residuals, mse_value)

print(f"MSE: {mse_value}")
print(f"Gaussian likelihood: {gaussian_likelihood_value}")


MSE: 0.009999999999999981
Gaussian likelihood: 82.94956719377812


In [5]:
from scipy.stats import laplace

def mean_absolute_error(y_true, y_pred):
    """Compute Mean Absolute Error (MAE)."""
    return np.mean(np.abs(y_true - y_pred))

def laplace_likelihood(residuals, scale):
    """Compute the Laplace likelihood of residuals."""
    n = len(residuals)
    likelihood = (1 / (2 * scale)) ** n * np.exp(-np.sum(np.abs(residuals)) / scale)
    return likelihood

# MAE and Laplace likelihood
mae_value = mean_absolute_error(y_true, y_pred)
laplace_likelihood_value = laplace_likelihood(residuals, mae_value)

print(f"MAE: {mae_value}")
print(f"Laplace likelihood: {laplace_likelihood_value}")


MAE: 0.09999999999999991
Laplace likelihood: 21.05608437214218


---
## Thank You for Exploring This Notebook!


If you have any questions, suggestions, or just want to discuss any of the topics further, please don't hesitate to reach out or leave a comment. Your feedback is not only welcome but also invaluable!

Happy analyzing, and stay curious!

---